In [4]:
import pandas as pd
import folium

## Get 2016 personal income data for Toronto area by census tract
1. Download Census tract data https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/prof/details/page_Download-Telecharger.cfm?Lang=E&Tab=1&Geo1=CSD&Code1=3520005&Geo2=PR&Code2=35&Data=Count&SearchText=Toronto&SearchType=Begins&SearchPR=01&B1=All&TABID=1
1. Unzip and save it in the same folder as this notebook (about 1.3Gb unzipped)
1. Run the following code to generate two files: income, and income as percent of Toronto (both files are by census tract)
1. Census tracts are filtered to Toronto area (those starting with 535)

We select `Median total income in 2015 among recipients ($)` variable.

**To calculate %, divide it by Toronto value of 30,089** (taken from https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/prof/details/page.cfm?Lang=E&Geo1=CSD&Geo2=PR&Code2=01&SearchType=Begins&SearchPR=01&TABID=1&B1=All&type=0&Code1=3520005&SearchText=toronto)

In [113]:
census2016 = pd.read_csv('./98-401-X2016043_English_CSV_data.csv', dtype={1:str, 3: str, 11: str})

In [148]:
# Extract `Median total income in 2015 among recipients ($)` variable
census2016_income = census2016[
        (census2016['DIM: Profile of Census Tracts (2247)'] == 'Median total income in 2015 among recipients ($)')
        & (census2016['GEO_CODE (POR)'].str.startswith('535'))
    ].filter(['GEO_CODE (POR)', 'Dim: Sex (3): Member ID: [1]: Total - Sex'])

# Rename columns
census2016_income.columns = ['Tract', 'Income']
census2016_income = census2016_income.set_index('Tract')

# Remove 'x' in income
census2016_income.Income = pd.to_numeric(census2016_income.Income, errors='coerce')

# Save (small file size, acceptable for GitHub)
census2016_income.to_csv('./census2016-income.csv', index=True)

# Save income as percent into a separate file
round(census2016_income / 30089 * 100, 1).to_csv('./census2016-income-as-percent.csv', index=True)

## Create a Leaflet map with `folium`

In [37]:
# Initialize map
m = folium.Map(
    location=[43.6529, -79.3849],
    tiles=None,
    zoom_start=12
)

# Add CartoDB Positron baselayer
folium.TileLayer(
    tiles='CartoDB positron',
    control=False,
    zoom_start=12
).add_to(m)

### Add Choropleth

In [38]:
# Read income data file for choropleth
census2016_income_percent = pd.Series( pd.read_csv('./census2016-income-as-percent.csv', dtype={0: str}).set_index('Tract').Income )

# Read census tracts GeoJSON
tracts = gpd.read_file('geodata/toronto-cma-tracts-2016-statcan-simplified.geojson').filter(['CTUID', 'geometry'])

# Attach Income column to tracts in order to be able to display it in a tooltip
tracts['Income'] = tracts.CTUID.apply(lambda x: census2016_income_percent.loc[x])

In [39]:
income_choropleth = folium.Choropleth(
    #geo_data=f'geodata/toronto-cma-tracts-2016-statcan-simplified.geojson',
    geo_data=tracts,
    name='Income',
    bins=[40, 60, 80, 100, 120, 140, 180, 270],
    data=census2016_income_percent,
    #columns=['Tract', 'Income'],
    key_on='feature.properties.CTUID',
    fill_color='RdBu',
    line_weight=1,
    line_color='white',
    nan_fill_color='silver',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Median Personal Income as % of Toronto",
    #overlay=True,
    highlight=True
)

# Add Tooltip data
folium.GeoJsonTooltip(fields=['CTUID', 'Income'], aliases=['Census Tract', 'Income as %']).add_to(income_choropleth.geojson)

# Add choropleth to the map
income_choropleth.add_to(m)

### Add Toronto+neighborhood overlays

In [40]:
# Toronto
folium.GeoJson(
    f'geodata/toronto-cma-boundary-2016-dli.geojson',
    name='Toronto',
    control=False,
    style_function=lambda x: {
        'fillOpacity': 0,
        'color': 'black',
        'weight': '1',
        'interactive': False
    }
).add_to(m)

# Etobicoke
folium.GeoJson(
    f'geodata/etobicoke-boundary-osm.geojson',
    name='Etobicoke',
    style_function=lambda x: {
        'fillOpacity': 0,
        'color': 'orange',
        'weight': '4',
        'interactive': False
    }
).add_to(m)

# Forrest Hill
folium.GeoJson(
    f'geodata/forest-hill-boundary-1961-dli.geojson',
    name='Forest Hill',
    style_function=lambda x: {
        'fillOpacity': 0,
        'color': 'purple',
        'weight': '4',
        'interactive': False
    }
).add_to(m)

# Scarborough
folium.GeoJson(
    f'geodata/scarborough-boundary-osm.geojson',
    name='Scarborough',
    style_function=lambda x: {
        'fillOpacity': 0,
        'color': 'green',
        'weight': '4',
        'interactive': False
    }
).add_to(m)

# York
folium.GeoJson(
    f'geodata/york-boundary-osm.geojson',
    name='York',
    style_function=lambda x: {
        'fillOpacity': 0,
        'color': 'gray',
        'weight': '4',
        'interactive': False
    }
).add_to(m)

### Final touches and save!

In [41]:
# Add layer control
folium.LayerControl(collapsed=False).add_to(m)

# And all done!
m.save('index.html')

## Manual additions to the generated HTML map

#### To add lines to the legend with neighbourhoods boundaries
```js
<span style='background:green;display:inline-block;vertical-align:middle;height:5px;width:25px;'></span>
```

#### To keep choropleth underneath neighbourhood boundaries
```js
choropleth_8926c1b6196e461e85cde667cbe9d1dd.on('add', function() {
  this.bringToBack();
}); 
```